We will try a more conceptual approach for label generation to step by step uncover which patterns can be captured by our community passing model

In [54]:
from utils import sbm_dataset, communityPassing
import pathpyG as pp
import torch

First of all we train with only zeroes as features and labels resemble Community affiliation

In [55]:
root= r"C:\Users\david\PythonProjekte\Bachelor\TestGraph"

dataset = sbm_dataset.StochasticBlockModelDataset(
    root=root,
    block_sizes=[5,6,7],
    edge_probs=[
        [0.7,0.05,0.05],
        [0.05,0.7,0.05],
        [0.05,0.05,0.7]
    ],
    is_undirected=True,
)
data = dataset[0]
data.y_task = data.y
data.x = torch.zeros(len(data.y), 1)

In [56]:
g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.y,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

In [57]:
from models.GCN import GCN
from utils.TrainingUtils import TrainingUtils

data.train_mask, data.val_mask, data.test_mask = TrainingUtils.make_splits(
    data.num_nodes,
    train_ratio=0.6,
    val_ratio=0.2,
    seed=123,
    device=data.x.device
)

model = GCN(
    in_channels=data.x.size(1),
    hidden=128,
    num_classes=3,
    dropout=0.35,
)

trained = model.fit(
    data,
    lr=3e-3,
    weight_decay=1e-3,
    epochs=400
)


Epoch 001 | loss 1.0986 | train 0.400 | val 0.667 | test 0.200
Epoch 020 | loss 1.0926 | train 0.400 | val 0.667 | test 0.200
Epoch 040 | loss 1.0897 | train 0.400 | val 0.667 | test 0.200
Epoch 060 | loss 1.0890 | train 0.400 | val 0.667 | test 0.200
Epoch 080 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 100 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 120 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 140 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 160 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 180 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 200 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 220 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 240 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 260 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 280 | loss 1.0889 | train 0.400 | val 0.667 | test 0.200
Epoch 300 | loss 1.0889 | train 0.400 | val 0.667 | tes

In [58]:
from models.communityGCN import CommunityGCN

model = CommunityGCN(
    in_channels=data.x.size(1),
    hidden=64,
    num_classes=3,
    dropout=0.2
)

trained = model.fit(
    data,
    community_attr="y",
    lr=1e-2,
    weight_decay=5e-4,
    epochs=200
)

Epoch 001 | loss 1.0742 | train 0.400 | val 0.667 | test 0.200
Epoch 020 | loss 1.0827 | train 0.400 | val 0.667 | test 0.200
Epoch 040 | loss 1.0654 | train 0.400 | val 0.667 | test 0.200
Epoch 060 | loss 1.0211 | train 0.400 | val 0.667 | test 0.200
Epoch 080 | loss 1.1053 | train 0.400 | val 0.667 | test 0.200
Epoch 100 | loss 1.0539 | train 0.400 | val 0.667 | test 0.200
Epoch 120 | loss 0.9406 | train 0.300 | val 0.000 | test 0.400
Epoch 140 | loss 1.2271 | train 0.400 | val 0.667 | test 0.200
Epoch 160 | loss 1.1760 | train 0.400 | val 0.667 | test 0.200
Epoch 180 | loss 1.0733 | train 0.400 | val 0.667 | test 0.200
Epoch 200 | loss 1.0455 | train 0.400 | val 0.667 | test 0.200
Best checkpoint (val) | train 0.400 | val 0.667 | test 0.200


In [59]:
def split_labels_in_two(y, seed=0):
    torch.manual_seed(seed)

    y = y.clone()
    new_y = torch.empty_like(y)

    for c in torch.unique(y):
        idx = (y == c).nonzero(as_tuple=True)[0]
        idx = idx[torch.randperm(len(idx))]

        half = len(idx) // 2
        new_y[idx[:half]] = 2 * c
        new_y[idx[half:]] = 2 * c + 1

    return new_y

In [60]:
data.y_task = split_labels_in_two(y=data.y)

In [61]:
g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.y_task,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

In [62]:
data.train_mask, data.val_mask, data.test_mask = TrainingUtils.make_splits(
    data.num_nodes,
    train_ratio=0.6,
    val_ratio=0.2,
    seed=123,
    device=data.x.device
)

model = GCN(
    in_channels=data.x.size(1),
    hidden=128,
    num_classes=6,
    dropout=0.35,
)

trained = model.fit(
    data,
    lr=3e-3,
    weight_decay=1e-3,
    epochs=400
)

Epoch 001 | loss 1.7918 | train 0.300 | val 0.000 | test 0.000
Epoch 020 | loss 1.7708 | train 0.300 | val 0.000 | test 0.000
Epoch 040 | loss 1.7528 | train 0.300 | val 0.000 | test 0.000
Epoch 060 | loss 1.7389 | train 0.300 | val 0.000 | test 0.000
Epoch 080 | loss 1.7282 | train 0.300 | val 0.000 | test 0.000
Epoch 100 | loss 1.7198 | train 0.300 | val 0.000 | test 0.000
Epoch 120 | loss 1.7133 | train 0.300 | val 0.000 | test 0.000
Epoch 140 | loss 1.7083 | train 0.300 | val 0.000 | test 0.000
Epoch 160 | loss 1.7046 | train 0.300 | val 0.000 | test 0.000
Epoch 180 | loss 1.7018 | train 0.300 | val 0.000 | test 0.000
Epoch 200 | loss 1.6998 | train 0.300 | val 0.000 | test 0.000
Epoch 220 | loss 1.6984 | train 0.300 | val 0.000 | test 0.000
Epoch 240 | loss 1.6974 | train 0.300 | val 0.000 | test 0.000
Epoch 260 | loss 1.6968 | train 0.300 | val 0.000 | test 0.000
Epoch 280 | loss 1.6964 | train 0.300 | val 0.000 | test 0.000
Epoch 300 | loss 1.6962 | train 0.300 | val 0.000 | tes

In [63]:
model = CommunityGCN(
    in_channels=data.x.size(1),
    hidden=64,
    num_classes=6,
    dropout=0.2
)

trained = model.fit(
    data,
    community_attr="y",
    lr=1e-2,
    weight_decay=5e-4,
    epochs=200
)

Epoch 001 | loss 1.7902 | train 0.200 | val 0.000 | test 0.200
Epoch 020 | loss 1.7250 | train 0.300 | val 0.000 | test 0.000
Epoch 040 | loss 1.7205 | train 0.300 | val 0.000 | test 0.000
Epoch 060 | loss 1.7182 | train 0.300 | val 0.000 | test 0.000
Epoch 080 | loss 1.6998 | train 0.300 | val 0.000 | test 0.000
Epoch 100 | loss 1.7017 | train 0.300 | val 0.000 | test 0.000
Epoch 120 | loss 1.6570 | train 0.300 | val 0.000 | test 0.000
Epoch 140 | loss 1.6513 | train 0.300 | val 0.000 | test 0.000
Epoch 160 | loss 1.6392 | train 0.300 | val 0.000 | test 0.000
Epoch 180 | loss 1.6738 | train 0.300 | val 0.000 | test 0.000
Epoch 200 | loss 1.6019 | train 0.300 | val 0.000 | test 0.000
Best checkpoint (val) | train 0.200 | val 0.000 | test 0.200


It looks like without expressive features the models arent learning antyhing. Next we will introduce very simple features that just resemble the labels of the nodes.

In [64]:
data.x = data.y.unsqueeze(1).float()
data.y_task = data.y

In [65]:
data.train_mask, data.val_mask, data.test_mask = TrainingUtils.make_splits(
    data.num_nodes,
    train_ratio=0.6,
    val_ratio=0.2,
    seed=123,
    device=data.x.device
)

model = GCN(
    in_channels=data.x.size(1),
    hidden=128,
    num_classes=3,
    dropout=0.35,
)

trained = model.fit(
    data,
    lr=3e-3,
    weight_decay=1e-3,
    epochs=400
)

Epoch 001 | loss 1.2501 | train 0.100 | val 0.000 | test 0.000
Epoch 020 | loss 0.8954 | train 0.400 | val 0.667 | test 0.200
Epoch 040 | loss 0.7931 | train 0.400 | val 0.667 | test 0.200
Epoch 060 | loss 0.7241 | train 0.600 | val 0.667 | test 0.600
Epoch 080 | loss 0.6042 | train 0.800 | val 1.000 | test 1.000
Epoch 100 | loss 0.4106 | train 1.000 | val 1.000 | test 1.000
Epoch 120 | loss 0.2468 | train 1.000 | val 1.000 | test 1.000
Epoch 140 | loss 0.1692 | train 1.000 | val 1.000 | test 1.000
Epoch 160 | loss 0.1035 | train 1.000 | val 1.000 | test 1.000
Epoch 180 | loss 0.0768 | train 1.000 | val 1.000 | test 1.000
Epoch 200 | loss 0.0584 | train 1.000 | val 1.000 | test 1.000
Epoch 220 | loss 0.0348 | train 1.000 | val 1.000 | test 1.000
Epoch 240 | loss 0.0371 | train 1.000 | val 1.000 | test 1.000
Epoch 260 | loss 0.0206 | train 1.000 | val 1.000 | test 1.000
Epoch 280 | loss 0.0099 | train 1.000 | val 1.000 | test 1.000
Epoch 300 | loss 0.0171 | train 1.000 | val 1.000 | tes

In [66]:
model = CommunityGCN(
    in_channels=data.x.size(1),
    hidden=64,
    num_classes=3,
    dropout=0.2
)

trained = model.fit(
    data,
    community_attr="y",
    lr=1e-2,
    weight_decay=5e-4,
    epochs=200
)

Epoch 001 | loss 1.1108 | train 0.900 | val 1.000 | test 1.000
Epoch 020 | loss 0.0829 | train 1.000 | val 1.000 | test 1.000
Epoch 040 | loss 0.0036 | train 1.000 | val 1.000 | test 1.000
Epoch 060 | loss 0.0011 | train 1.000 | val 1.000 | test 1.000
Epoch 080 | loss 0.0018 | train 1.000 | val 1.000 | test 1.000
Epoch 100 | loss 0.0011 | train 1.000 | val 1.000 | test 1.000
Epoch 120 | loss 0.0038 | train 1.000 | val 1.000 | test 1.000
Epoch 140 | loss 0.0010 | train 1.000 | val 1.000 | test 1.000
Epoch 160 | loss 0.0029 | train 1.000 | val 1.000 | test 1.000
Epoch 180 | loss 0.0010 | train 1.000 | val 1.000 | test 1.000
Epoch 200 | loss 0.0034 | train 1.000 | val 1.000 | test 1.000
Best checkpoint (val) | train 0.900 | val 1.000 | test 1.000


This seems to work well, but it also just a direct dependency between the labels and the features, so that is kind of expected. We will try again by splitting the clusters up.

In [67]:
data.y_task = split_labels_in_two(y=data.y)

In [ ]:
model = GCN(
    in_channels=data.x.size(1),
    hidden=128,
    num_classes=6,
    dropout=0.35,
)

trained = model.fit(
    data,
    lr=3e-3,
    weight_decay=1e-3,
    epochs=400
)

pred = trained(data.x, data.edge_index).argmax(dim=1)

Epoch 001 | loss 1.7854 | train 0.100 | val 0.667 | test 0.200
Epoch 020 | loss 1.5412 | train 0.300 | val 0.000 | test 0.000
Epoch 040 | loss 1.4242 | train 0.400 | val 0.000 | test 0.000
Epoch 060 | loss 1.3551 | train 0.500 | val 0.000 | test 0.200
Epoch 080 | loss 1.1919 | train 0.600 | val 0.000 | test 0.400
Epoch 100 | loss 1.0227 | train 0.700 | val 0.000 | test 0.400
Epoch 120 | loss 0.8580 | train 0.700 | val 0.000 | test 0.400
Epoch 140 | loss 0.7329 | train 0.800 | val 0.000 | test 0.400
Epoch 160 | loss 0.6693 | train 0.800 | val 0.000 | test 0.400
Epoch 180 | loss 0.5493 | train 0.800 | val 0.000 | test 0.400
Epoch 200 | loss 0.5099 | train 0.800 | val 0.000 | test 0.400
Epoch 220 | loss 0.4536 | train 0.800 | val 0.000 | test 0.400
Epoch 240 | loss 0.4236 | train 0.800 | val 0.000 | test 0.400
Epoch 260 | loss 0.4043 | train 0.800 | val 0.000 | test 0.400
Epoch 280 | loss 0.3883 | train 0.800 | val 0.000 | test 0.400
Epoch 300 | loss 0.3943 | train 0.800 | val 0.000 | tes

In [69]:
g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Node Labels:")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.y_task,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Node Features")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.x,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Label Predictions:")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=pred,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

Node Labels:


Node Features


Label Predictions:


In [73]:
model = CommunityGCN(
    in_channels=data.x.size(1),
    hidden=64,
    num_classes=6,
    dropout=0.2
)

trained = model.fit(
    data,
    community_attr="y",
    lr=1e-2,
    weight_decay=5e-4,
    epochs=200
)

pred = trained(data.x, data.edge_index, data.y).argmax(dim=1)

Epoch 001 | loss 2.1476 | train 0.300 | val 0.000 | test 0.000
Epoch 020 | loss 0.7608 | train 0.800 | val 0.000 | test 0.400
Epoch 040 | loss 0.4309 | train 0.800 | val 0.000 | test 0.400
Epoch 060 | loss 0.2894 | train 0.900 | val 0.000 | test 0.400
Epoch 080 | loss 0.1818 | train 0.900 | val 0.333 | test 0.400
Epoch 100 | loss 0.1643 | train 1.000 | val 0.000 | test 0.200
Epoch 120 | loss 0.1322 | train 1.000 | val 0.000 | test 0.200
Epoch 140 | loss 0.1595 | train 1.000 | val 0.000 | test 0.200
Epoch 160 | loss 0.0906 | train 0.900 | val 0.000 | test 0.400
Epoch 180 | loss 0.0806 | train 1.000 | val 0.000 | test 0.200
Epoch 200 | loss 0.1317 | train 1.000 | val 0.000 | test 0.200
Best checkpoint (val) | train 0.500 | val 0.667 | test 0.600


In [75]:
g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Node Labels:")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.y_task,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Node Features")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.x,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Label Predictions:")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=pred,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

Node Labels:


Node Features


Label Predictions:


Both models didn't learn properly, but it is very obvious, that the community Passing model captured the excact distribution of the node features for the node labels aswell, whereas the Vanilla GCN didn't. \
Therefore we will now also split up the features in the same way we did for the labels and see if this will still hold up.

In [ ]:
data.x = data.y_task.unsqueeze(1).float()

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]])
tensor([[0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [2.],
        [2.],
        [3.],
        [3.],
        [3.],
        [2.],
        [5.],
        [5.],
        [5.],
        [4.],
        [5.],
        [4.],
        [4.]])


In [78]:
model = GCN(
    in_channels=data.x.size(1),
    hidden=128,
    num_classes=6,
    dropout=0.35,
)

trained = model.fit(
    data,
    lr=3e-3,
    weight_decay=1e-3,
    epochs=400
)

pred = trained(data.x, data.edge_index).argmax(dim=1)

Epoch 001 | loss 1.9462 | train 0.100 | val 0.000 | test 0.200
Epoch 020 | loss 1.6323 | train 0.300 | val 0.000 | test 0.000
Epoch 040 | loss 1.4618 | train 0.400 | val 0.000 | test 0.000
Epoch 060 | loss 1.3601 | train 0.500 | val 0.000 | test 0.200
Epoch 080 | loss 1.1913 | train 0.500 | val 0.000 | test 0.200
Epoch 100 | loss 1.0771 | train 0.700 | val 0.000 | test 0.400
Epoch 120 | loss 0.9007 | train 0.700 | val 0.000 | test 0.400
Epoch 140 | loss 0.8227 | train 0.700 | val 0.000 | test 0.400
Epoch 160 | loss 0.7425 | train 0.800 | val 0.000 | test 0.400
Epoch 180 | loss 0.6369 | train 0.800 | val 0.000 | test 0.400
Epoch 200 | loss 0.5847 | train 0.800 | val 0.000 | test 0.400
Epoch 220 | loss 0.5613 | train 0.800 | val 0.000 | test 0.400
Epoch 240 | loss 0.5055 | train 0.800 | val 0.000 | test 0.400
Epoch 260 | loss 0.4635 | train 0.800 | val 0.000 | test 0.400
Epoch 280 | loss 0.4069 | train 0.900 | val 0.000 | test 0.400
Epoch 300 | loss 0.4455 | train 0.800 | val 0.000 | tes

In [79]:
g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Node Labels:")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.y_task,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Node Features")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.x,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Label Predictions:")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=pred,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

Node Labels:


Node Features


Label Predictions:


In [80]:
model = CommunityGCN(
    in_channels=data.x.size(1),
    hidden=64,
    num_classes=6,
    dropout=0.2
)

trained = model.fit(
    data,
    community_attr="y",
    lr=1e-2,
    weight_decay=5e-4,
    epochs=200
)

pred = trained(data.x, data.edge_index, data.y).argmax(dim=1)

Epoch 001 | loss 2.2204 | train 0.300 | val 0.000 | test 0.000
Epoch 020 | loss 0.8020 | train 0.800 | val 0.000 | test 0.400
Epoch 040 | loss 0.5673 | train 0.800 | val 0.000 | test 0.400
Epoch 060 | loss 0.3697 | train 0.800 | val 0.000 | test 0.400
Epoch 080 | loss 0.2872 | train 0.900 | val 0.333 | test 0.400
Epoch 100 | loss 0.2836 | train 0.900 | val 0.333 | test 0.400
Epoch 120 | loss 0.2592 | train 0.900 | val 0.000 | test 0.200
Epoch 140 | loss 0.1511 | train 0.900 | val 0.333 | test 0.400
Epoch 160 | loss 0.1747 | train 0.900 | val 0.000 | test 0.400
Epoch 180 | loss 0.1455 | train 0.900 | val 0.333 | test 0.400
Epoch 200 | loss 0.1333 | train 0.900 | val 0.000 | test 0.400
Best checkpoint (val) | train 0.800 | val 0.333 | test 0.400


In [81]:
g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Node Labels:")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.y_task,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Node Features")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=data.x,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

g = pp.Graph.from_edge_index(edge_index=data.edge_index, num_nodes=len(data.x))

print("Label Predictions:")
pp.plot(
    g.to_undirected(),     
    backend="d3js",

    layout="kk",           
    node_color=pred,           
    cmap="tab10",

    #node_size=16,
    #edge_opacity=0.15,
    # edge_size=1,

    show_labels=False,
)

Node Labels:


Node Features


Label Predictions:


We can observer the vanilla gcn couldn't capture this dependency at all and assigned the same label to each node. The community Passing Algorithm on the other hand was able to detect the community structures in the data, even with the mixed up features within, therefore it was able to at least get half of the nodes right.